In [1]:
import numpy as np
import os
import sys
import cv2
import matplotlib.pyplot as plt
import open3d as o3d
%matplotlib inline 

src_path = os.path.abspath("../..")
if src_path not in sys.path:
    sys.path.append(src_path)
%load_ext autoreload
from dataset.kitti_odometry_dataset import KittiOdometryDataset, KittiOdometryDatasetConfig
from dataset.filters.filter_list import FilterList
from dataset.filters.kitti_gt_mo_filter import KittiGTMovingObjectFilter
from dataset.filters.range_filter import RangeFilter
from dataset.filters.apply_pose import ApplyPose

import scipy
from scipy.spatial.distance import cdist
from normalized_cut import normalized_cut
from ncuts_utils import ncuts_chunk,kDTree_1NN_feature_reprojection_colors, get_merge_pcds
from dataset_utils import * 
from point_cloud_utils import get_pcd, transform_pcd, kDTree_1NN_feature_reprojection, remove_isolated_points, get_subpcd, get_statistical_inlier_indices, merge_chunks_unite_instances, merge_chunks_unite_instances2
from aggregate_pointcloud import aggregate_pointcloud
from visualization_utils import generate_random_colors, color_pcd_by_labels,generate_random_colors_map
from sam_label_distace import sam_label_distance
from chunk_generation import subsample_positions, chunks_from_pointcloud, indices_per_patch, tarl_features_per_patch, image_based_features_per_patch, dinov2_mean, get_indices_feature_reprojection
from metrics.metrics_class import Metrics
import shutil
from open3d.pipelines import registration
from predict_maskpls import RefinerModel

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


/home/cedric/anaconda3/envs/torch/lib/python3.9/site-packages/MinkowskiEngine-0.5.4-py3.9-linux-x86_64.egg/MinkowskiEngine/__init__.py:36: UserWarning: The environment variable `OMP_NUM_THREADS` not set. MinkowskiEngine will automatically set `OMP_NUM_THREADS=16`. If you want to set `OMP_NUM_THREADS` manually, please export it on the command line before running a python script. e.g. `export OMP_NUM_THREADS=12; python your_program.py`. It is recommended to set it below 24.
  warnings.warn(


Here we define the dataset depending on kitti sequence!

In [2]:
DATASET_PATH = os.path.join('/media/cedric/Datasets2/semantic_kitti/')
end_inds = {0:4541,1:1100,2:4661,3:800,4:271,5:2761,6:1101,7:1100,8:4071,9:1591,10:1201}
SEQUENCE_NUM = 9

ind_start = 0
ind_end = end_inds[SEQUENCE_NUM]

minor_voxel_size = 0.05
major_voxel_size = 0.35
chunk_size = np.array([25, 25, 25]) #meters
overlap = 3 #meters
ground_segmentation_method = 'patchwork' 
NCUT_ground = False

out_chunks = 'pcd_preprocessed/output_chunks/'


dataset = create_kitti_odometry_dataset(DATASET_PATH,SEQUENCE_NUM,ncuts_mode=True)

out_folder = 'pcd_preprocessed/'
if os.path.exists(out_folder) == False : 
        os.makedirs(out_folder)


Now we aggregate a large point cloud based on (ind_start, ind_end)
## This cell can be ignored after first run as outputs are stored 

This cell can be ignored after first run as outputs are stored 

Now we subsample the poses based on a voxel_size

In [3]:
out_kitti_instance = out_chunks + 'test_data' + str(SEQUENCE_NUM) + '/' 
#out_kitti_instance = out_chunks + 'out_refined/'
        
out_refined = out_chunks + 'out_refined'  + str(SEQUENCE_NUM) + '/' 
if os.path.exists(out_refined) == True : 
        shutil.rmtree(out_refined)
os.makedirs(out_refined)


maskpls = RefinerModel()

out_data = []

center_ids = os.listdir(out_kitti_instance)
for sequence in range(len(center_ids)):
        if NCUT_ground == False : 
                
                name = center_ids[sequence]
                print("sequence",sequence)
                cur_pcd = o3d.io.read_point_cloud(out_kitti_instance + name)
                predicted_refined = maskpls.forward_and_project(cur_pcd)
                #o3d.visualization.draw_geometries([predicted_refined]) 
                #maskpls.project_pseudo_labels(cur_pcd,center_ids[sequence],name)
                o3d.io.write_point_cloud(out_refined + name, predicted_refined,write_ascii=False, compressed=False, print_progress=False)
                
                        #pcd_dbscan = DBSCAN_clustering_logic(cur_pcd,
                        #                        eps=0.3, min_samples=10)
                        #o3d.io.write_point_cloud(out_dbscan + name, pcd_dbscan + pcd_chunk_ground, write_ascii=False, compressed=False, print_progress=False)
                        
                
                
                #kitti_chunk_instance_ground = color_pcd_by_labels(pcd_chunk_ground,kitti_labels['ground']['instance'][sequence].reshape(-1,))
                #o3d.io.write_point_cloud(out_dbscan + name, pcd_dbscan + pcd_chunk_ground, write_ascii=False, compressed=False, print_progress=False)
                

        
        
        

using weights /media/cedric/Datasets21/Weights/KITTI/TARL_Spatial_Dino/mask_pls_oversegmented_epoch=05.ckpt
sequence 0
sequence 1
sequence 2
sequence 3
sequence 4
sequence 5
sequence 6
sequence 7
sequence 8
sequence 9
sequence 10
sequence 11
sequence 12
sequence 13
sequence 14
sequence 15
sequence 16
sequence 17
sequence 18
sequence 19
sequence 20
sequence 21
sequence 22
sequence 23
sequence 24
sequence 25
sequence 26
sequence 27
sequence 28
sequence 29
sequence 30
sequence 31
sequence 32
sequence 33
sequence 34
sequence 35
sequence 36
sequence 37
sequence 38
sequence 39
sequence 40
sequence 41
sequence 42
sequence 43
sequence 44
sequence 45
sequence 46
sequence 47
sequence 48
sequence 49
sequence 50
sequence 51
sequence 52
sequence 53
sequence 54
sequence 55
sequence 56
sequence 57
sequence 58
sequence 59
sequence 60
sequence 61
sequence 62
sequence 63
sequence 64
sequence 65
sequence 66
sequence 67
sequence 68
sequence 69
sequence 70
sequence 71
sequence 72
sequence 73
sequence 74


In [4]:
def get_merge_pcds(out_folder_ncuts):
        point_clouds = []

        # List all files in the folder
        files = os.listdir(out_folder_ncuts)
        files.sort()

        # Filter files with a .pcd extension
        pcd_files = [file for file in files if file.endswith(".pcd")]
        print(pcd_files)
        # Load each point cloud and append to the list
        for pcd_file in pcd_files:
                file_path = os.path.join(out_folder_ncuts, pcd_file)
                point_cloud = o3d.io.read_point_cloud(file_path)
                point_clouds.append(point_cloud)
        return point_clouds


def merge_unite_gt(chunks):
    last_chunk = chunks[0] 
    merge = o3d.geometry.PointCloud()
    merge += last_chunk

    for new_chunk in chunks[1:]:
        merge += new_chunk
    
    merge.remove_duplicated_points()
    return merge 

def intersect(pred_indices, gt_indices):
        intersection = np.intersect1d(pred_indices, gt_indices)
        return intersection.size / pred_indices.shape[0]


def remove_semantics(labels,preds):
        gt_idcs = np.where(labels == 0)[0]
        new_ncuts_labels = preds.copy()
        for i in np.unique(preds):
                pred_idcs = np.where(preds == i)[0]
                cur_intersect = intersect(pred_idcs,gt_idcs)
                if cur_intersect > 0.8:
                        new_ncuts_labels[pred_idcs] = 0
        return new_ncuts_labels

def remove_semantics_pcd(pcd,labels,preds):
        cols = np.asarray(pcd.colors)
        gt_idcs = np.where(labels == 0)[0]
        new_ncuts_labels = preds.copy()
        for i in np.unique(preds):
                pred_idcs = np.where(preds == i)[0]
                cur_intersect = intersect(pred_idcs,gt_idcs)
                if cur_intersect > 0.8:
                        new_ncuts_labels[pred_idcs] = 0
                        cols[pred_idcs] = np.array([0,0,0])
                        
        pcd.colors = o3d.utility.Vector3dVector(cols)   
        return pcd




In [5]:
out_kitti = out_chunks + 'out_kitti/'
out_kitti_instance = out_chunks + 'out_kitti_instance/'


point_clouds_refined = get_merge_pcds(out_refined)[:-1]

#merge = merge_chunks_unite_instances(point_clouds)
#merge_dbscan = merge_chunks_unite_instances(point_clouds_dbscan)
#merge_kitti = merge_unite_gt(point_clouds_kitti)
#merge_kitti_instance = merge_unite_gt(point_clouds_kitti_instances)
merge_refined = merge_chunks_unite_instances2(point_clouds_refined)

#o3d.io.write_point_cloud(out_folder + "merge_part.pcd", merge, write_ascii=False, compressed=False, print_progress=False)
o3d.io.write_point_cloud(out_folder + "merge_refined_test" + str(SEQUENCE_NUM) + ".pcd", merge_refined, write_ascii=False, compressed=False, print_progress=False)
#o3d.io.write_point_cloud(out_folder + "merge_part_dbscan.pcd", merge_dbscan, write_ascii=False, compressed=False, print_progress=False)
#o3d.io.write_point_cloud(out_folder + "merge_part_kitti.pcd", merge_kitti, write_ascii=False, compressed=False, print_progress=False)
#o3d.io.write_point_cloud(out_folder + "merge_part_kitti_instance.pcd", merge_kitti_instance, write_ascii=False, compressed=False, print_progress=False)

#unique_colors, labels_ncuts = np.unique(np.asarray(merge.colors), axis=0, return_inverse=True)



['000044.pcd', '000068.pcd', '000089.pcd', '000109.pcd', '000128.pcd', '000146.pcd', '000166.pcd', '000189.pcd', '000214.pcd', '000235.pcd', '000252.pcd', '000267.pcd', '000282.pcd', '000298.pcd', '000317.pcd', '000339.pcd', '000362.pcd', '000385.pcd', '000409.pcd', '000432.pcd', '000454.pcd', '000474.pcd', '000494.pcd', '000516.pcd', '000540.pcd', '000561.pcd', '000581.pcd', '000601.pcd', '000623.pcd', '000649.pcd', '000677.pcd', '000703.pcd', '000728.pcd', '000749.pcd', '000768.pcd', '000786.pcd', '000802.pcd', '000818.pcd', '000835.pcd', '000856.pcd', '000878.pcd', '000898.pcd', '000918.pcd', '000946.pcd', '000977.pcd', '001007.pcd', '001030.pcd', '001052.pcd', '001074.pcd', '001100.pcd', '001133.pcd', '001156.pcd', '001177.pcd', '001197.pcd', '001215.pcd', '001232.pcd', '001248.pcd', '001264.pcd', '001280.pcd', '001296.pcd', '001312.pcd', '001328.pcd', '001344.pcd', '001360.pcd', '001376.pcd', '001392.pcd', '001407.pcd', '001422.pcd', '001437.pcd', '001452.pcd', '001467.pcd', '0014

100%|██████████| 73/73 [21:45<00:00, 17.89s/it]


True

In [6]:
#new_ncuts_labels = remove_semantics(labels_kitti,labels_ncuts)
merge_kitti_instance = o3d.io.read_point_cloud(out_folder + "merge_part_kitti_instance" + str(SEQUENCE_NUM) +  ".pcd")
merge_refined = o3d.io.read_point_cloud(out_folder + 'merge_refined_test' + str(SEQUENCE_NUM) + '.pcd')
unique_colors, labels_kitti = np.unique(np.asarray(merge_kitti_instance.colors),axis=0, return_inverse=True)
unique_colors, labels_refined = np.unique(np.asarray(merge_refined.colors),axis=0, return_inverse=True)
print(labels_kitti.shape,labels_refined.shape)
new_refined = remove_semantics_pcd(merge_refined,labels_kitti,labels_refined)

(38238324,) (38238324,)


In [7]:
unique_colors, labels_refined = np.unique(np.asarray(new_refined.colors),axis=0, return_inverse=True)
label_to_confidence = {}
pcd_cols = np.asarray(new_refined.colors) 
for label in list(np.unique(labels_refined)):
	idcs = np.where(labels_refined == label)[0]
	cur_color = pcd_cols[idcs[0]]
	key = str(int(cur_color[0] * 255)) + '|' + str(int(cur_color[1] * 255)) + '|' + str(int(cur_color[2] * 255))
	label_to_confidence[label] = maskpls.confs_dict[key].cpu().item()

In [8]:
metrics_refined = Metrics(name='refined')
unique_colors, orig_labels = np.unique(np.asarray(merge_refined.colors),axis=0, return_inverse=True)

metrics_refined.update_stats(orig_labels,labels_refined,labels_kitti,label_to_confidence,calc_all=True)


start
Metrics for file refined
{'panoptic': 0.6716540813270094, 'precision': 0.7934782608695652, 'recall': 0.7227722772277227, 'fScore': 0.7564766839378239, 'usr': 0.021739130434782608, 'osr': 0.039603960396039604, 'noise': 0.16304347826086957, 'missed': 0.25742574257425743, 'mean': 0.8878714910692659}
S_assoc score :  0.6604410348328336


In [ ]:
o3d.visualization.draw_geometries([merge_refined])